In [73]:
import pandas as pd
from sklearn.neighbors import BallTree
import geopandas as gpd
from shapely.geometry import Point
from fuzzywuzzy import fuzz

c:\Users\ahmad\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv(r'../data/pm_osm_addresses_geocodes.csv')

In [3]:
cbsa_data = pd.read_csv(r'../data/cbsa_osm_addresses.csv')

C:\Users\ahmad\AppData\Local\Temp\ipykernel_25852\2760806279.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  cbsa_data = pd.read_csv(r'../data/cbsa_osm_addresses.csv')


In [7]:
df

,Unnamed: 0.1,Unnamed: 0,company_name,building_name,raw_address,author,building_name_and_address,osm_address,osm_latitude,osm_longitude
0,0,0,Air Communities,"Upton Place\r\n4000 Wisconsin Ave NW, Washingt...","4000 Wisconsin Ave NW, Washington, DC 20016",Patrick,"Upton Place 4000 Wisconsin Ave NW, Washington,...","Onelife Fitness, 4000, Wisconsin Avenue Northw...",38.941589,-77.076599
1,1,1,Air Communities,Ravel & Royale on Strathmore Square\r\n10511 S...,"10511 Strathmore Hall St, North Bethesda, MD 2...",Patrick,Ravel & Royale on Strathmore Square 10511 Stra...,"Ravel and Royale at Strathmore Hall, 10511, St...",39.028720,-77.101672
2,2,2,Air Communities,"Palazzo East Apartments\r\n348 S Hauser Blvd, ...","348 S Hauser Blvd, Los Angeles, CA 90036",Patrick,"Palazzo East Apartments 348 S Hauser Blvd, Los...","Hauser Boulevard, Park La Brea, Fairfax, Los A...",34.066431,-118.350293
3,3,3,Air Communities,"Flats 8300\r\n8300 Wisconsin Ave, Bethesda, MD...","8300 Wisconsin Ave, Bethesda, MD 20814",Patrick,"Flats 8300 8300 Wisconsin Ave, Bethesda, MD 20814","Flats 8300, 8300, Wisconsin Avenue, Wisconsin ...",38.993420,-77.096950
4,4,4,Air Communities,"Township Residences\r\n901 E Phillips Ln, Cent...","901 E Phillips Ln, Centennial, CO 80122",Patrick,"Township Residences 901 E Phillips Ln, Centenn...","901, East Phillips Lane, Township Residences, ...",39.568911,-104.977605
...,...,...,...,...,...,...,...,...,...,...
8819,8819,8819,Willow Bridge Property Co.,NaN,"6400 Sycamore Lane Maple Grove, MN",Chelle,"6400 Sycamore Lane Maple Grove, MN","6400, Sycamore Lane North, Maple Grove, Hennep...",45.068620,-93.443127
8820,8820,8820,Willow Bridge Property Co.,NaN,"16255 Kenyon Ave. Lakeville, MN",Chelle,"16255 Kenyon Ave. Lakeville, MN","16255, Kenyon Avenue, Lakeville, Dakota County...",44.713735,-93.286212
8821,8821,8821,Willow Bridge Property Co.,NaN,"800 Walnut Place Chaska, MN",Chelle,"800 Walnut Place Chaska, MN","800, Walnut Place, Chaska, Carver County, Minn...",44.791949,-93.601458
8822,8822,8822,Willow Bridge Property Co.,NaN,"13905 Chestnut Drive Eden Prairie, MN",Chelle,"13905 Chestnut Drive Eden Prairie, MN","Chestnut Drive, Eden Prairie, Hennepin County,...",44.851579,-93.454782


In [10]:
gdf_pm = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df['osm_longitude'], df['osm_latitude'])],
    crs='EPSG:4326'
)

In [11]:
gdf_cbsa = gpd.GeoDataFrame(
    cbsa_data,
    geometry=[Point(xy) for xy in zip(cbsa_data['longitude'], cbsa_data['latitude'])],
    crs='EPSG:4326'
)

In [12]:
joined = gpd.sjoin_nearest(gdf_cbsa, gdf_pm, how="left", distance_col="distance")

c:\Users\ahmad\anaconda3\lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [22]:
joined[['full_address','cbsa_osm_address','raw_address','osm_address','distance']].iloc[0]

full_address                     2400 Arrowhead Dr, Abilene, TX 79606
cbsa_osm_address    2400, Arrowhead Drive, Abilene, Taylor County,...
raw_address                    3649 Cedar Run Road, Abilene, TX 79606
osm_address         3649, Cedar Run Road, Abilene, Taylor County, ...
distance                                                     0.021199
Name: 0, dtype: object

In [19]:
cbsa_data.columns

Index(['Unnamed: 0', 'propertyId', 'propertyName', 'zipCode', 'address',
       'city', 'state', 'averageSquareFootage', 'stories', 'stable', 'class',
       'latitude', 'longitude', 'imageHero', 'heroSource', 'submarketName',
       'unitcount', 'yearBuilt', 'siteStatus', 'type', 'MPF-ANN-RENT-CHG',
       'MPF-HIST-ASK-RENT', 'MPF-HIST-AVG-ASK-RPSF', 'MPF-HIST-CONC-RATIO',
       'MPF-OCC', 'MPF-RENT', 'MPF-RPSF', 'data_index', 'cbsa_code', 'status',
       'SH-HIST-ASK-RENT-PB', 'SH-HIST-RENT-PB', 'SH-HIST-CONC-RATIO-PB',
       'SH-HIST-OCCUPANCY', 'SH-HIST-RPSF-PB', 'SH-HIST-AVG-ASK-RPSF-PB',
       'SH-HIST-ANN-RENT-PB-CHG', 'state zip', 'full_address',
       'cbsa_osm_address'],
      dtype='object')

In [17]:
joined.columns

Index(['Unnamed: 0_left', 'propertyId', 'propertyName', 'zipCode', 'address',
       'city', 'state', 'averageSquareFootage', 'stories', 'stable', 'class',
       'latitude', 'longitude', 'imageHero', 'heroSource', 'submarketName',
       'unitcount', 'yearBuilt', 'siteStatus', 'type', 'MPF-ANN-RENT-CHG',
       'MPF-HIST-ASK-RENT', 'MPF-HIST-AVG-ASK-RPSF', 'MPF-HIST-CONC-RATIO',
       'MPF-OCC', 'MPF-RENT', 'MPF-RPSF', 'data_index', 'cbsa_code', 'status',
       'SH-HIST-ASK-RENT-PB', 'SH-HIST-RENT-PB', 'SH-HIST-CONC-RATIO-PB',
       'SH-HIST-OCCUPANCY', 'SH-HIST-RPSF-PB', 'SH-HIST-AVG-ASK-RPSF-PB',
       'SH-HIST-ANN-RENT-PB-CHG', 'state zip', 'full_address',
       'cbsa_osm_address', 'geometry', 'index_right', 'Unnamed: 0.1',
       'Unnamed: 0_right', 'company_name', 'building_name', 'raw_address',
       'author', 'building_name_and_address', 'osm_address', 'osm_latitude',
       'osm_longitude', 'distance'],
      dtype='object')

In [15]:
joined[joined['distance'] > 2]

,Unnamed: 0_left,propertyId,propertyName,zipCode,address,city,state,averageSquareFootage,stories,stable,...,Unnamed: 0_right,company_name,building_name,raw_address,author,building_name_and_address,osm_address,osm_latitude,osm_longitude,distance
613,613,8840,Country Club Villas,79109,4401 Coulter St S,Amarillo,TX,791,3,stable,...,5695,Kairoi Residential,The Woodford on Mockingbird,"1412 Mockingbird Ln, Midland, TX 79705",Tim,The Woodford on Mockingbird 1412 Mockingbird L...,"1412, Mockingbird Lane, Adobe Meadows, Midland...",32.061591,-102.074750,3.105432
614,614,8841,The Aspen,79109,4615 S Virginia St,Amarillo,TX,848,3,stable,...,5695,Kairoi Residential,The Woodford on Mockingbird,"1412 Mockingbird Ln, Midland, TX 79705",Tim,The Woodford on Mockingbird 1412 Mockingbird L...,"1412, Mockingbird Lane, Adobe Meadows, Midland...",32.061591,-102.074750,3.105302
615,615,9676,Chasewood,79109,3420 S Coulter,Amarillo,TX,773,3,stable,...,5695,Kairoi Residential,The Woodford on Mockingbird,"1412 Mockingbird Ln, Midland, TX 79705",Tim,The Woodford on Mockingbird 1412 Mockingbird L...,"1412, Mockingbird Lane, Adobe Meadows, Midland...",32.061591,-102.074750,3.117292
616,616,11464,Huntington Pointe,79106,6801 Wolflin Ave,Amarillo,TX,790,3,stable,...,5695,Kairoi Residential,The Woodford on Mockingbird,"1412 Mockingbird Ln, Midland, TX 79705",Tim,The Woodford on Mockingbird 1412 Mockingbird L...,"1412, Mockingbird Lane, Adobe Meadows, Midland...",32.061591,-102.074750,3.133321
617,617,13974,Indian Trail,79109,4120 Prairie Ave,Amarillo,TX,819,2,stable,...,5695,Kairoi Residential,The Woodford on Mockingbird,"1412 Mockingbird Ln, Midland, TX 79705",Tim,The Woodford on Mockingbird 1412 Mockingbird L...,"1412, Mockingbird Lane, Adobe Meadows, Midland...",32.061591,-102.074750,3.105674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38361,38361,380960,The Flats of Acme Village,49690,5541 Foothills Dr,Williamsburg,MI,1417,1,NaN,...,8130,Highmark Residential,STERLING CREST APARTMENTS,"1307 TITTABAWASSEE RD,\r\nSAGINAW, MI 48604",Ahmad,STERLING CREST APARTMENTS 1307 TITTABAWASSEE R...,"1307, Tittabawassee Road, Zilwaukee, Carrollto...",43.479565,-83.942319,2.019530
39029,39029,164414,Rivercrest,83301,2005 Rivercrest Dr,Twin Falls,ID,1034,3,NaN,...,4806,Greystar Management Services,"One West\r\n2112 W 3300 S, West Haven, UT 84401","2112 W 3300 S, West Haven, UT 84401",Patrick,"One West 2112 W 3300 S, West Haven, UT 84401","West 3300 South, West Haven, Weber County, Uta...",41.205339,-112.079547,2.770064
39030,39030,376644,Elizabeth Estates,83301,1854 Elizabeth Blvd,Twin Falls,ID,1189,2,NaN,...,4806,Greystar Management Services,"One West\r\n2112 W 3300 S, West Haven, UT 84401","2112 W 3300 S, West Haven, UT 84401",Patrick,"One West 2112 W 3300 S, West Haven, UT 84401","West 3300 South, West Haven, Weber County, Uta...",41.205339,-112.079547,2.726577
41472,41472,310180,The Marketplace,57201,15 1st St NW,Watertown,SD,1122,4,NaN,...,7640,Weidner Property Management,The Club at Heritage,"720 Roberts Road, Sartell, MN 56377",Ahmad,"The Club at Heritage 720 Roberts Road, Sartell...","720, Roberts Road, Sartell, Stearns County, Mi...",45.601174,-94.217733,2.982076


In [23]:
#change metrics to use meters instead of kilometers
gdf_pm_metric = gdf_pm.to_crs(epsg=3857)
gdf_cbsa_metric = gdf_cbsa.to_crs(epsg=3857)

In [56]:
joined2 = gpd.sjoin_nearest(
    gdf_cbsa_metric, 
    gdf_pm_metric, 
    how="left", 
    max_distance=150,
    distance_col="distance"
)

In [77]:
matched_prop = joined2[joined2['distance'].isna() == False]

In [66]:
print(joined2[joined2['distance'] > 100][['full_address','cbsa_osm_address','raw_address','osm_address','distance']].iloc[9])

full_address                 3608 Ladera Dr NW, Albuquerque, NM 87120
cbsa_osm_address    3608, Ladera Drive Northwest, Albuquerque, Ber...
raw_address               3704 Ladera Drive NW, Albuquerque, NM 87120
osm_address         3704, Ladera Drive Northwest, Albuquerque, Ber...
distance                                                   147.706286
Name: 434, dtype: object


In [68]:
joined2[joined2['distance'] > 100]['osm_address'].iloc[9]

'3704, Ladera Drive Northwest, Albuquerque, Bernalillo County, New Mexico, 87120, United States'

In [69]:
joined2[joined2['distance'] > 100]['full_address'].iloc[9]

'3608 Ladera Dr NW, Albuquerque, NM 87120'

In [70]:
joined2[joined2['distance'] > 100].iloc[9]

Unnamed: 0_left                                                            434
propertyId                                                              102845
propertyName                                           Ladera Vista Apartments
zipCode                                                                  87120
address                                                      3608 Ladera Dr NW
city                                                               Albuquerque
state                                                                       NM
averageSquareFootage                                                       735
stories                                                                      3
stable                                                                  stable
class                                                                        A
latitude                                                             35.123614
longitude                                           

In [72]:
joined2[joined2['distance'] > 100]['company_name'].iloc[9]

'Mission Rock Residential'

In [75]:
similarity_threshold = 80
def match_names(row):
    if pd.isnull(row.get('building_name')):
        return 0
    similarity = fuzz.token_set_ratio(row['propertyName'], row['building_name'])
    return 1 if similarity >= similarity_threshold else 0

In [79]:
matched_prop['name_match_score'] = matched_prop.apply(match_names, axis=1)

c:\Users\ahmad\anaconda3\lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [96]:
geo_code_list = matched_prop[matched_prop['name_match_score'] == 1]['propertyId'].unique()

In [82]:
fuz_match_results = pd.read_csv(r'../data/fuzzy_matched_results_full.csv')

In [ ]:
fuzzy_match_list = fuz_match_results[fuz_match_results['match_score'] > 75]['propertyId'].unique()

In [104]:
joined2['fuzz_match_check'] = joined2['propertyId'].isin(fuzzy_match_list)
joined2['geocode_match_check'] = joined2['propertyId'].isin(geo_code_list)

In [116]:
joined2[(joined2['geocode_match_check'] == False) & (joined2['fuzz_match_check'] == True) & (joined2['osm_longitude'].notnull())].iloc[0]

Unnamed: 0_left                                                           1561
propertyId                                                               20900
propertyName                                                 Aspire Lenox Park
zipCode                                                                  30319
address                                                1050 Lenox Park Blvd NE
city                                                                   Atlanta
state                                                                       GA
averageSquareFootage                                                      1020
stories                                                                      4
stable                                                                  stable
class                                                                        B
latitude                                                             33.851223
longitude                                           

In [115]:
joined2[(joined2['geocode_match_check'] == False) & (joined2['fuzz_match_check'] == True) & (joined2['osm_longitude'].notnull())]['raw_address'].iloc[0]

'1020 Lenox Park Blvd NE, Atlanta, GA 30319'

In [114]:
joined2[(joined2['geocode_match_check'] == False) & (joined2['fuzz_match_check'] == True) & (joined2['osm_longitude'].notnull())]['full_address'].iloc[0]

'1050 Lenox Park Blvd NE, Atlanta, GA 30319'